In [1]:
#from serial import Serial
import serial
import matplotlib.pyplot as plt
import time
import numpy as np


s_sensor = serial.Serial(port = "COM5", baudrate=115200,bytesize=8, timeout=2, stopbits=serial.STOPBITS_ONE)
s_printer = serial.Serial(port = "COM4", baudrate=250000)


"""Mecmesin Advanced Force Gauge Code"""
s_AFG = serial.Serial(port = "COM10", baudrate=115200,bytesize=8, timeout=2, stopbits=serial.STOPBITS_ONE)
s_AFG.write(0x2A)#* 42 0x2A Continuous transmit
def read_force():
    """Reads the force from Mecmesin AFG on s_AFG"""
    time.sleep(0.05)
    s_AFG.flushInput()
    s_AFG.write(0x3F)#? 63 0x3F Transmit the current reading
    force_N = s_AFG.readline().decode().strip()
    time.sleep(0.05)
    s_AFG.flushInput()
    s_AFG.write(0x3F)#? 63 0x3F Transmit the current reading
    force_N = s_AFG.readline().decode().strip()
    while force_N == '':#Incase empty bit arrives
        s_AFG.flushInput()
        s_AFG.write(0x3F)#? 63 0x3F Transmit the current reading
        force_N = s_AFG.readline().decode().strip()
    force_N = abs(float(force_N))#AFG reports compression as negative
    #print(force_N)
    return force_N

In [2]:
s_piezo = serial.Serial(port = "COM3", baudrate=9600)
def read_force():
    s_piezo.flushInput()
    force_IDK = s_piezo.readline().decode().strip()
    while force_IDK == "":
        force_IDK = s_piezo.readline().decode().strip()
    return int(force_IDK)

In [3]:
read_force()

0

In [4]:
def read_printer():
    """Reads printers current position over serial"""
    s_printer.flushInput()
    s_printer.write("M114 R\n".encode())#M114 returns cooridnated
    grbl_out = s_printer.readline().decode()
    #print(grbl_out)
    while(grbl_out[0] != "X"):#Sometimes temperature readings get transmitted instead
        grbl_out = s_printer.readline().decode()
    #print(grbl_out)
    data = grbl_out.split()[0:3]#X, Y, Z
    data = [float(d[2:]) for d in data]
    return data

In [5]:
def read_sensor():
    """Reads ReSkin Sensor from arduino over serial"""
    s_sensor.flushInput()
    serialString = s_sensor.readline()
    serialString = serialString.decode('Ascii')
    #time.sleep(0.001)
    
    b20 = [float(b) for b in serialString.split()]
    return b20

In [6]:
def setpos(X,Y,Z):
    """Sets position of 3d printer via serial (Marlin)"""
    line = "G0 "+ " X"+str(X)+" Y"+str(Y)+ " Z"+str(Z)+  " F" + str(feedrate)+"\n"
    s_printer.write(line.encode()) # Send g-code block to grbl
    line = "M400 \n"
    s_printer.write(line.encode())#M400 halts gcode until move is completed
    line = "M118 moved!\n"
    s_printer.write(line.encode()) #M118 asks serial to send from 3d (but only after M400/move is completed)
    grbl = s_printer.readline().decode()
    while grbl != 'moved!\r\n':#Dont continue if we have not moved
        grbl = s_printer.readline().decode()
    return

In [7]:
feedrate = "1600"

In [8]:
#Setup
# Wake up grbl
s_printer.write("\r\n\r\n".encode())
time.sleep(2)   # Wait for grbl to initialize
s_printer.flushInput()  # Flush startup text in serial input

#Set established 0/0/0 pos
"""Sets origin for printer - set position to be above bottom left screw and 2mm above sensor surface (X & Y + 10)"""


print('Sending: ' + "G90")
s_printer.write("G90\n".encode())

#setpos(x_def, y_def, z_def)
print(read_printer())
print('Sending: ' + "G92")
s_printer.write("G92 X0 Y0 Z0\n".encode())
print(read_printer())
time.sleep(3)

Sending: G90
[0.0, 0.0, -1.6]
Sending: G92
[0.0, 0.0, 0.0]


In [9]:
read_printer()

[0.0, 0.0, 0.0]

In [10]:
setpos(0,0,0)

In [11]:
b20 = read_sensor()
print(b20)

[60.15, -105.3, -45.5, 28.72, 60.6, -147.6, -49.13, 27.96, -664.05, 931.35, -812.39, 26.9, -68.55, 2258.85, 152.7, 26.17, -89.1, 80.55, -100.19, 28.38]


In [ ]:
lin = np.arange(2,3.8,0.01)
setpos(10,10,0)
forces = []
for depth in lin:
    setpos(10,10,-depth)
    forces.append(read_force())
    time.sleep(0.1)
setpos(10,10,0)
plt.plot(lin,forces)
plt.xlabel("Depth [mm]")
plt.ylabel("Response [v]")
plt.title("Force Sensor Response")
plt.show()

In [ ]:
"""Normalization Values collected before and after"""
norm_data = []
setpos(0,0,0)
time.sleep(1)

for i in range(5000):
    b20 = read_sensor()
    if i%1000 == 0:
        print(i)
        print("B20 Read")
        print(b20)
    norm_data.append(b20)

In [ ]:
print(len(norm_data))

In [ ]:
"""Avoid corners with screws"""
notest = []
for i in (0,1,7,8):
    for q in (0,1,7,8):
        notest.append([i,q])
notest.append([0,5,3])
print(len(notest))
print(notest)
print([0,5] in notest) 

In [ ]:
truths = []
sensor_data = []

feedrate = 1000
setpos(0,0,0)

In [ ]:
iterations = 250
#print(f"Estimated time to completion: {round(170*iterations/60,0)}min")
grid_x = 9
grid_y = 9
jump_mm = 19/9
z_depths = [0.2,0.4,0.6,0.8,1,1.2,1.4]
z_depths_mm = [-z-2.4 for z in z_depths]
wait_time = 0.5
setpos(0,0,0)

no_cord = []
time_start = time.time()
time_for_iteration = 0
for iteration in range(1,iterations):
    setpos(0,0,0)
    for g_x in range(grid_x):#Iteratate over grid
        g_x_mm = round(g_x*jump_mm,2)
        setpos(g_x_mm,0,0)
        for g_y in range(grid_y):
            if not ([g_x,g_y] in notest): #Remove corners with screws
                g_y_mm = round(g_y*jump_mm,2)
                setpos(g_x_mm,g_y_mm,0)#move above testing depth
                for z_mm in z_depths_mm:
                    setpos(g_x_mm,g_y_mm,z_mm)#move to testing depth

                    #Readdata
                    b20 = read_sensor()
                    sensor_data.append(b20)

                    force_N = read_force()
                    if force_N == 0:#If not touching sensor we define as 0/0/0
                        #notest.append([g_x,g_y,z_mm])
                        truths.append([-1,-1,force_N])
                        #print("Notest:",[g_x,g_y,z_mm,z_mm])
                    else:
                        truths.append([g_x_mm,g_y_mm,force_N])
                        #print([g_x_mm,g_y_mm,force_N])
                else:
                    b20 = read_sensor()
                    sensor_data.append(b20)
                    truths.append([-1,-1,0])
                setpos(g_x_mm,g_y_mm,0)#Move above sensor again
    
    time_for_iteration = round(time.time()-time_start,0)-time_for_iteration
    if iteration == 1:
        print("Initial time: ", time_for_iteration)
        print(f"Initial time to completion: {round(time_for_iteration*(iterations-iteration)/60,0)}min")
    print(f"Iterations: {iteration}/{iterations}")
    #print(f"Current iter time: {time_for_iteration}s")
    #print([g_x_mm,g_y_mm,force_N])
    #print(b20)
setpos(0,0,0)

 


In [ ]:
print(notest)

In [ ]:
print(f"Total time: {round(time.time()-time_start,0)}s")
print(f"Time/Iteration: {round((time.time()-time_start)/iterations,0)}s")
print(f"Sensor Data count: {len(sensor_data)}")
print(f"Truth count: {len(truths)}")

sensor_data = sensor_data[:len(truths)]

print(f"Sensor Data count: {len(sensor_data)}")
print(f"Truth count: {len(truths)}")

In [ ]:
#Normalization values:
setpos(0,0,0)
time.sleep(5)

for i in range(5000):
    b20 = read_sensor()
    if i%1000 == 0:
        print(i)
        print("B20 Read")
        print(b20)
    norm_data.append(b20)
print(len(norm_data))

In [ ]:
print(f"Sensor Data count: {len(sensor_data)}")
print(sensor_data[0:3])
print(f"Truth count: {len(truths)}")
print(truths[0:3])
print(f"Norm Value count: {len(norm_data)}")
print(norm_data[0:3])

In [ ]:
np.savetxt("C:/Users/timku/Dropbox/Neural Computation Project/Models/Data/norm_b20_artillery_piezo_screw.txt", norm_data, fmt="%s")
np.savetxt("C:/Users/timku/Dropbox/Neural Computation Project/Models/Data/b20_artillery_piezo_screw.txt", sensor_data, fmt="%s")
np.savetxt("C:/Users/timku/Dropbox/Neural Computation Project/Models/Data/truths_artillery_piezo_screw.txt", truths, fmt="%s")

truths =  np.loadtxt("C:/Users/timku/Dropbox/Neural Computation Project/Models/Data/truths_artillery_screw.txt",dtype = float)
sensor_data = np.loadtxt("C:/Users/timku/Dropbox/Neural Computation Project/Models/Data/b20_artillery_screw.txt",dtype = float)
norm_data = np.loadtxt("C:/Users/timku/Dropbox/Neural Computation Project/Models/Data/norm_b20_artillery_screw.txt",dtype = float) 

truths2 =  np.loadtxt("C:/Users/timku/Dropbox/Neural Computation Project/Models/Data/truths_artillery_screw2.txt",dtype = float)
sensor_data2 = np.loadtxt("C:/Users/timku/Dropbox/Neural Computation Project/Models/Data/b20_artillery_screw2.txt",dtype = float)
norm_data2 = np.loadtxt("C:/Users/timku/Dropbox/Neural Computation Project/Models/Data/norm_b20_artillery_screw2.txt",dtype = float) 

truths = np.concatenate((truths,truths2))
sensor_data  = np.concatenate((sensor_data,sensor_data2 ))
norm_data = np.concatenate((norm_data,norm_data2))
print(len(truths))